In [ ]:
import cv2
import os
import numpy as np
import h5py
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.models import Model

class DataGenerator:
    def __init__(self, dataset_dir):
        self.video_files = sorted([entry.name for entry in os.scandir(os.path.join(dataset_dir, 'videos')) if entry.is_file()], key=str.lower)
        self.dataset_dir = dataset_dir
        self.feature_dir = os.path.join(self.dataset_dir, 'NASNetLarge_feature_vectors.hdf5')
        self.nasnet_large = self.initialize_nasnet_large()

        if not os.path.exists(self.feature_dir):
            with h5py.File(self.feature_dir, 'w') as hf:
                pass  

    @staticmethod
    def initialize_nasnet_large():
        print("Initializing NASNetLarge model...")
        full_model = NASNetLarge(weights='imagenet', include_top=True)
        return Model(inputs=full_model.input, outputs=full_model.get_layer('normal_concat_18').output)

    def process_video(self, video_file):
        try:
            print(f"Processing video: {video_file}")
            video = cv2.VideoCapture(os.path.join(self.dataset_dir, 'videos', video_file))
            video_features = []

            frame_count = 0
            while True:
                ret, frame = video.read()
                if not ret:
                    break
                try:
                    print(f"Processing frame {frame_count} in video: {video_file}")
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = cv2.resize(frame, (331, 331))  
                    frame = preprocess_input(frame)
                    frame_features = self.nasnet_large.predict(np.expand_dims(frame, axis=0))
                    video_features.append(frame_features)

                    frame_count += 1
                except Exception as e:
                    print(f"Error processing frame {frame_count} in video {video_file}: {e}")

            video.release()

            with h5py.File(self.feature_dir, 'a') as hf:
                hf.create_dataset(video_file, data=np.array(video_features))

            print(f"Completed processing video: {video_file}")

        except Exception as e:
            print(f"Error processing video {video_file}: {e}")

if __name__ == "__main__":
    data_generator = DataGenerator('./train')

    print("Starting video processing...")

    for video_file in data_generator.video_files:
        data_generator.process_video(video_file)
    
    print("Video processing completed.")
